# Week 07: Statistics, part 1

This week's learning goals are as follows:

1. Learn the difference between population statistics and sample statistics.
1. Understand the Normal/Gaussian distribution and how it relates to sampling.
1. t-testing. 

This notebook uses the Kaggle Dataset Pokemon with stats. Download and move the csv into ```07_statistics/csvs```. For this notebook, I have defined a set of util functions for working with our Pokemon data. Please copy over the ```pokemon_util.py``` file from Week 4.

This notebook also uses the MyAnimeList database, and I've defined util functions for that as well. Please download the csv from [this GitHub repo](https://github.com/Dibakarroy1997/myanimelist-data-set-creator), "MyAnimeList Anime Dataset up to May 7 2018" and save it in ```07_statistics/csvs```. I have defined a set of util functions for working with the MAL data. Please copy over the ```mal_util.py``` file from Week 5.

In [ ]:
# the following code guarantees you'll properly reload any modules that you custom-defined in your environment.
# you don't need to understand it.
# just run this once at the beginning.
# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import csv
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
from pokemon_util import *
pokemon_fpath = os.path.join('csvs', 'pokemon.csv')
poke_headers, poke_types, poke_dict = load_pokemon(pokemon_fpath) # get the dictionary
poke_arr, poke_np_lookup = poke_array(poke_dict, poke_types) # convert into numpy array
print('pokemon headers', poke_headers)

from mal_util import *
mal_fpath = os.path.join('csvs', 'myAnimeListDataset [07-05-2018].csv')
anime_headers, anime_studios, anime_sources, anime_dict = load_mal(mal_fpath)
anime_arr, anime_np_lookup = anime_array(anime_dict,  anime_studios, anime_sources) # convert into numpy array
print('anime headers', anime_headers)

## 1. Population vs sample

Imagine for a second you want to characterize a population -- say, Pokemon from generation III. However, maybe you have not caught them all, and instead you have a smattering of Pokemon from everyone in Gen III. The Pokemon you have in your box is what is called a **sample**, i.e., a sequence of Pokemon that are taken from the original population. Notice it is possible for samples to contain the same element multiple times.

In [ ]:
def make_sample(arr, sample_size):
    num_elts = arr.shape[0]
    inds = np.random.randint(num_elts, size=sample_size)
    return arr[inds, :]

print('sample of sample size 20:')
print(make_sample(poke_arr, 20))



If you only have a *sample*, all statistics you compute are actually with respect to the Pokemon in your sample, not with respect to the entire population. Therefore if you take the mean and variance of your Pokemon sample, you are not guaranteed that they are equal to the mean and variance of your entire population.

In [ ]:
def sample_means(headers, arr,
                    stat_ind, sample_size,
                 num_samples, print_indiv=True):
    print('Overall {}: mean {}, var {} (std {})'.format(
        headers[stat_ind],
        np.average(arr[:,stat_ind]),
            '%.2f' % np.var(arr[:,stat_ind]), '%.2f' % np.std(arr[:,stat_ind])))
    sample_stats = []
    for i in range(num_samples):
        sample = make_sample(arr, sample_size)[:,stat_ind]
        sample_stats.append(np.average(sample))
        if print_indiv:
            print('sample {}: mean {}, var {} (std {})'.format(
                i+1,
                '%.2f' % np.average(sample),
                '%.2f' % np.var(sample),
                '%2f' % np.std(sample)))
    print('Average of {} sample means (each of size {}): {}'.format(
        num_samples, sample_size, np.average(sample_stats)))
    
sample_size = 20
num_samples = 1
stat_ind = TOTAL_STAT
sample_means(poke_headers, poke_arr,
             stat_ind, sample_size,
             num_samples, print_indiv=True)

The **sample mean** is a variable, which depends on the particular items in your given sample.

The population mean \\( \mu \\) is defined as follows:

\\[ \mu = \frac{1}{n} \sum_{k=1}^n x_k  \\]

Whereas the sample mean \\( \bar{x} \\) is only defined across the samples \\( x_k \\) that are in the sample to begin with. However, it can be proved that *on average*, the sample mean converges to the population mean.

In [ ]:
num_samples = 300
sample_means(poke_headers, poke_arr,
             stat_ind, sample_size,
             num_samples, print_indiv=False)

The population variance \\( \sigma^2 \\) is defined as the average squared distance from the population mean:

\\[ \sigma^2 = \frac{1}{n} \sum_{k=1}^n (x_k - \mu)^2  \\]

The **sample variance** is also a variable. The sample variance is defined as follows, where \\( x_j \\) are now those that are in our sample, of sample size \\(m \\).

\\[ S^2 = \frac{1}{m-1} \sum_{j=1}^{m} (x_j - \bar{x})^2  \\]

Notice that the denominator is slightly different than the population variance. We say that the sample variance has one fewer **degree of freedom** than the sample mean; this is mostly because the squared distance is with respect to the sample mean, not the population mean. In any case, using a denominator of \\( m - 1\\) allows us to ensure that *on average*, the sample variance converges to the population variance, we need to reduce our degree of freedom by 1. The proof for this is a bit complicated but follows from the linearity of averages.

The **sample standard deviation** is the square root of the sample variance:


\\[ S = \sqrt{\frac{1}{m-1} \sum_{j=1}^{m} (x_j - \bar{x})^2  }\\]

It's a bit difficult to show that the sample variance is *unbiased* (reaches the population population variance in the average), because our Pokemon dataset is so skewed, but here is an attempt:

In [ ]:
def sample_stdevs(headers, arr,
                    stat_ind, sample_size,
                 num_samples, print_indiv=True):
    print('Overall {}: mean {}, var {}'.format(
        headers[stat_ind],
        np.average(arr[:,stat_ind]), np.std(arr[:,stat_ind])))
    sample_stats = []
    for i in range(num_samples):
        sample = make_sample(arr, sample_size)[:,stat_ind]
        sample_stats.append(np.std(sample)) # straight up stdev
        sample_stats.append(np.std(sample, ddof=1)) # actual sample stdev
    print('Average of {} sample statistics (each of size {}): regular stdev {}, sample stdev {}'.format(
        num_samples, sample_size,
        np.average(sample_stats[0]), np.average(sample_stats[1])))

num_samples = 1000
sample_size = 50
sample_stdevs(poke_headers, poke_arr,
             stat_ind, sample_size,
             num_samples, print_indiv=False)

### Standard error (SE)

The **standard error** is different from the standard deviation. The standard deviation is the average deviation of an element's value from the population mean. However, the standard error is the average deviation of the *sample mean* from the actual population mean. In other words, if you considered all possible sample means from samples of size 20, how much (on average) would a sample mean deviate from the actual population mean?

Recall that the average of sample means is the population mean itself. The standard error \\( \sigma_x \\) is defined as the sample standard deviation divided by the square root of \\(m \\), the number of values in the sample:
 
\\[ \sigma_x \approx \frac{S}{\sqrt{m}} \\]

Why is this true? It can be shown in general that \\( \sigma_x^2 = \text{Var}(\frac{1}{m} \sum_{k=1}^m X_k) = \frac{1}{m} \text{Var}(X_k) \\), where \\(\text{Var}(X_k) = \sigma^2 \\), the population variance and \\( X_k \\) are independent. Therefore \\( \sigma_x = \frac{1}{\sqrt{m}} \sigma^2 \\). However, since the population standard deviation \\(\sigma \\) is not known in general, we approximate \\( \sigma^2 \approx S \\), the sample standard deviation.

In [ ]:
from scipy import stats
def sample_SEs(headers, arr,
                    stat_ind, sample_size,
                 num_samples, print_indiv=True):
    print('Overall {}: mean {}, var {}'.format(
        headers[stat_ind],
        np.average(arr[:,stat_ind]), np.std(arr[:,stat_ind])))
    sample_stats = []
    for i in range(num_samples):
        sample = make_sample(arr, sample_size)[:,stat_ind]
        sample_stats.append(np.average(sample))
        sample_stats.append(stats.sem(sample))
    print('Average of {} sample statistics (each of size {}): average {}, standard error {}'.format(
        num_samples, sample_size,
        '%.2f' % np.average(sample_stats[0]),
        '%.2f' % np.average(sample_stats[1])))

num_samples = 1000
sample_size = 50
sample_SEs(poke_headers, poke_arr,
             stat_ind, sample_size,
             num_samples, print_indiv=False)

This discussion of standard deviation and standard error then begs the question -- if we would like to report some measure of variance/spread of our experiment statistics, which one should we report? And the answer depends on what you are reporting.

1. Your experiment is on the entire population

    If you are reporting the **entire population**, your average is not a sample mean; it is a population mean, and therefore you should report the variance. It therefore does not make sense to include any sense of error bars, since that's all you have, and you should really be reporting the population standard deviation as a value in your paper.
    
1. Your experiment is on a subset of the entire population
    If you are reporting a **sample** of the entire population --- for example, you only were able to operate on a sample size of 50 in a potentially infinitely large population -- then your statistics are with respect to the sample.
       
    You often want to make claims about the population as a whole (and how they would be affected by said experiment). So you report the average (i.e., sample mean) of the effect, and you claim that the population average would be very similar to the sample mean average, but you keep in mind that there is some error (i.e., standard error) of your claimed average versus what the real average is.
    
Keep in mind that in general, we almost *never* know the full population. So we are almost always reporting on the sample and generalizing to the population, which is why we report standard error.

In [ ]:
sample_size = 20
stat_ind = TOTAL_STAT
num_samples = 3

sample_stats = []
for i in range(num_samples):
    sample = make_sample(poke_arr, sample_size)[:,stat_ind]
    sample_stats.append( (np.average(sample), np.std(sample), stats.sem(sample)) )

# incorrect
fig = plt.figure()
ax = plt.gca()
sample_stats = np.array(sample_stats)
bincenters = np.linspace(0.5, num_samples+0.5, num_samples)
ax.bar(bincenters, sample_stats[:,0], width=0.8, alpha=0.8)
# plotting error bars
ax.errorbar(bincenters, sample_stats[:,0], yerr=sample_stats[:,1],capsize=3, fmt='none', color='k')
ax.set_xticks(bincenters)
ax.set_xticklabels(['Sample {}'.format(i+1) for i in range(num_samples)])
ax.set_ylabel('Average {} Stat'.format(poke_headers[stat_ind]))
ax.set_title('Incorrect error bars (sample stdev instead of SE)')
    
# correct
fig = plt.figure()
ax = plt.gca()
sample_stats = np.array(sample_stats)
bincenters = np.linspace(0.5, num_samples+0.5, num_samples)
# can combine error bars with ax.bar() call
ax.bar(bincenters, sample_stats[:,0], width=0.8, alpha=0.8, yerr=sample_stats[:,2],capsize=3)
ax.set_xticks(bincenters)
ax.set_xticklabels(['Sample {}'.format(i+1) for i in range(num_samples)])
ax.set_ylabel('Average {} Stat'.format(poke_headers[stat_ind]))
ax.set_title('Correct error bars')

#### Programming exercises

1. Report the following statistics on the MAL dataset (population):
    1. Mean score
    1. Standard deviation of score
    
1. Take 100 samples (of size 20) of the MAL dataset and report:
    1. Sample mean score
    1. Sample standard deviation of score
    1. Average sample mean score
    1. Average sample standard deviation pf scpre
    1. Average standard error in score

1. Plot a histogram of the estimated average number of ratings (with error bars) of three samples (of size 20).

As a reminder of this dataset:

In [ ]:
print('anime array shape', anime_arr.shape)
anime_headers

## 2. The normal distribution

In general, we do not have multiple samples of the population; we have exactly one sample, and we need to generalize the population mean. So we report the sample mean coupled with standard error.

In fact, we can go further with this meaning of standard error. We first begin by noting that the distribution of sample means is Gaussian (i.e., Normal), meaning it has a bell-curve. So all samples (of a given sample size) are distributed normally.

Here is an example with the average of numbers between 0 and 1:

In [ ]:
def random_sample_means(sample_size, num_samples):
    sample_arr = np.zeros((num_samples, 3))
    for i in range(num_samples):
        sample = np.random.rand(sample_size)
        sample_arr[i, :] = (np.average(sample), np.std(sample), stats.sem(sample))
    return np.array(sample_arr)
    
sample_size = 50
num_samples = 10000
stat_ind = TOTAL_STAT
sample_arr = random_sample_means(sample_size, num_samples)
fig = plt.figure()
ax = plt.gca()
ax.hist(sample_arr[:,0],bins=20,alpha=0.8)
ax.set_title('Sample means of 50 random numbers between 0 and 1.')

Don't believe me? Here's the distribution of sample means of anime scores.

In [ ]:
def mal_sample_means(sample_size, num_samples):
    sample_arr = np.zeros((num_samples, 3))
    num_elts = anime_arr.shape[0]
    for i in range(num_samples):
        inds = np.random.randint(num_elts, size=sample_size)
        sample = anime_arr[inds,SCORE]
        sample_arr[i, :] = (np.average(sample), np.std(sample), stats.sem(sample))
    return np.array(sample_arr)
    
sample_size = 50
num_samples = 100000
stat_ind = TOTAL_STAT
sample_arr = mal_sample_means(sample_size, num_samples)
fig = plt.figure()
ax = plt.gca()
ax.hist(sample_arr[:,0],bins=20,alpha=0.8)
ax.set_xlim((5, 8))
ax.set_title('Sample means of 50 random anime scores.')

What does this mean for reporting statistics? This means that the general rule of thumb for reporting **confidence intervals** on a Gaussian distribution hold. For example, a 95% confidence interval means that we are 95% confidence that the true population mean falls within 1.96 standard errors of our sample mean.

The proof is left here for my documentation, where \\( m\\) is our sample size, \\( \frac{S}{\sqrt{m}} \\) is the standard error, and \\( \bar{x} \\) is the sample mean in question. We are calculating for 1.96 standard deviations from the mean, corresponding to a 95 % confidence interval.

\\[ P( \mu - 1.96 \frac{S}{\sqrt{m}} \leq \bar{x} \leq \mu + 1.96 \frac{S}{\sqrt{m}} ) = 0.95\\]

\\[ = P(- 1.96 \frac{S}{\sqrt{m}} \leq \bar{x} - \mu \leq 1.96 \frac{S}{\sqrt{m}} )\\]

\\[ = P( - 1.96 \frac{S}{\sqrt{m}} -  \bar{x} \leq - \mu \leq 1.96 \frac{S}{\sqrt{m}} -  \bar{x}) \\]


\\[ = P( \bar{x} + 1.96 \frac{S}{\sqrt{m}} \geq \mu \geq \bar{x} - 1.96 \frac{S}{\sqrt{m}}) \\]


In [ ]:
sample_size = 1000
inds = np.random.randint(anime_arr.shape[0], size=sample_size)
single_sample = anime_arr[inds,SCORE]
fig = plt.figure()
ax = plt.gca()
ax.hist(sample,bins=50)
ax.set_title('A random 1000-size sample of MAL scores from the MAL distribution')
bar_x = np.average(single_sample)
se = stats.sem(single_sample)
print('sample mean {}, standard error {}'.format(
'%.2f' % bar_x, '%.2f' % se))
print('We are 95% confident that our true population mean')
print('\tlies within the interval {}±{}, i.e., ({}, {})'.format(
    '%.2f' % bar_x, '%.2f' % (se*1.96),
    '%.2f' % (bar_x - 1.96*se), '%.2f' % (bar_x + 1.96*se)
))
print("True population mean: {}".format('%.2f' % np.average(anime_arr[:,SCORE])))

#### Programming exercises

Take the following exponential distribution:

In [ ]:
exp_values = stats.expon.rvs(scale=5, size=1000)
plt.hist(values,bins=50)
plt.show()

1. Do the following 10,000 times:
    1. Take a random sample (sample size 50) of exponentially distributed random variables (with scale=5). You can generate this by calling ```stats.expon.rvs(scale=5, size=50)```.
    1. Take the sample mean of this sample and save it.
1. Plot the distribution of sample means (i.e., histogram) and show that it is a normal distribution.

In [ ]:
# your code here

## 4. Using t-tests

Sometimes you want to compare two samples to each other. Traditionally, the most common way to do this is via a **t-test**, though nowadays it is more correct to return **bootstrapped** values (which we'll discuss next time). But let's talk about a t-test for now.

Suppose you want to measure:
* Whether an experiment had an effect (compare control group to experimental group)
* Whether two anime seasons have significantly different average scores

In both of these cases, if we assume the underlying population distributions are Gaussian (normal), then we want to show via the t-test that the two sample groups that we have are from different (normally distributed) populations. We do so by proving that the **null hypothesis** -- that is, the case where the two samples are from the same distribution -- has a very low probability. The claim that the two samples are from different distributions is called the **alternate hypothesis**.

Null hypotheses of the two groups above:
* The experiment has no effect (both groups are random samples from the same population)
* The two anime seasons have scores that are drawn from a general population of all anime

There are **many** caveats of the t-test! The first is that the distribution that you want to measure is a normal distribution. This is very obviously not the case in Pokemon, but it is the case in MAL scores per anime (because actually the distribution of MAL scores is a distribution of averages, and from the normal discussion above we saw that distributions of averages tend towards a normal distribution). See the population distributions below.

The second caveat of the t-test is that it assumes that all items in the sample are independent. This is not true in Pokemon, because stat levels are dependent on evolution, so two randomly selected Pokemon in the same evolutionary tree will have similar stats. In MAL, the independence guarantee also breaks down slightly; we have many sequels of anime (e.g., Gintama), so some scores are dependent on others. 


In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,4))
axs[0].hist(poke_arr[:,TOTAL_STAT],bins=50)
axs[0].set_title('Distribution of total Pokemon stat')
axs[1].hist(anime_arr[:,SCORE], bins=50)
axs[1].set_title('Distribution of MAL anime scores')
plt.show()

The steps to perform a t-test (if we didn't have Python) are as follows:
1. Assume the null hypothesis
1. Calculate the t value, which is some function of the difference between the sample means and the population mean of the difference
1. Map the t value to the p value
1. If the p value is below the threshold for statistical significance, reject the null hypothesis in favor of the alternate hypothesis.

What does "the threshold for statistical significance" mean? Generally, this is p < 0.05 for pleb scientists, and **p < 0.001** for true significance.

The steps to perform a t-test on samples a and b when we have Python are as follows:
1. ```from scipy import stats```
1. ```stats.ttest_ind(a, b, equal_var=True/False)```

Python/SciPy documentation for the t-test is [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html).

Anyway let's perform a t-test. Let's take samples (sample size 20) from two different studios and see whether sample mean of the scores significantly differ.

In [ ]:
from scipy import stats

studio1, studio2 = 'A-1 Pictures', 'J.C.Staff'
studio1_anime = anime_arr[anime_arr[:,STUDIO] == anime_studios.index(studio1),:]
studio2_anime = anime_arr[anime_arr[:,STUDIO] == anime_studios.index(studio2),:]
sample_size = 20
sample1_inds = np.random.randint(studio1_anime.shape[0], size=sample_size)
sample2_inds = np.random.randint(studio2_anime.shape[0], size=sample_size)

scores1 = studio1_anime[sample1_inds, SCORE]
scores2 = studio2_anime[sample2_inds, SCORE]
t, p = stats.ttest_ind(scores1, scores2, equal_var=False)
print(p)

This is not a significant statistic (for many thresholds of p), so we do not have enough evidence to reject the null hypothesis.

On the other hand:
* Null hypothesis: Toei Animation and Bones have scores that come from the same distribution.
* Alternate hypothesis: Toei Animation and Bones have significantly different scores.

In [ ]:
studio1, studio2 = 'Toei Animation', 'Bones'
studio1_anime = anime_arr[anime_arr[:,STUDIO] == anime_studios.index(studio1),:]
studio2_anime = anime_arr[anime_arr[:,STUDIO] == anime_studios.index(studio2),:]
sample_size = 20
sample1_inds = np.random.randint(studio1_anime.shape[0], size=sample_size)
sample2_inds = np.random.randint(studio2_anime.shape[0], size=sample_size)

scores1 = studio1_anime[sample1_inds, SCORE]
scores2 = studio2_anime[sample2_inds, SCORE]
t, p = stats.ttest_ind(scores1, scores2, equal_var=False)
print(p)

This p-value is significant (p < 0.001) , we will get a p-value that is significant enough for us to reject the null hypothesis. So we can conclude that the samples of anime that we have from Toei and from Bones are significantly different.

In [ ]:
studio_info = [(studio, anime_arr[anime_arr[:,STUDIO] == anime_studios.index(studio),:].shape[0]) \
               for studio in anime_studios]
studio_info = sorted(studio_info,key=lambda x: -x[1])
for x in studio_info:
    print(x)

#### Programming exercises

Find two anime studios that have significantly different number of ratings.

In [ ]:
# your code here

## 5. Homework

1. Graph the distribution of sample means of the MAL scores from a season of your choice.
1. For two different seasons of anime, report via results of the two-sided t-test if two anime seasons are significantly different in average MAL score.
1. Choose three different studios and sample 10 anime from each. Graph a histogram of the average score per studio, with error bars.